In [14]:
import base64
import datetime
import json
import os
import random
import time

import scipy
from scipy import ndimage
import numpy as np
import pandas as pd
import sklearn
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import rcParams
import seaborn as sns
import statsmodels as sm
from pylab import rcParams
from pylab import *
from matplotlib.dates import date2num , DateFormatter
from PIL import Image

np.random.seed(1337)

%matplotlib inline
sns.set(font_scale=1.0)
rcParams['figure.figsize'] = 8, 6
sns.set_style('whitegrid')
sns.set_palette(sns.color_palette('muted'))

In [15]:
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer

# Load the Data

In [16]:
d = pd.read_json('tmp/raw_data.json')
d.index.name = 'id'
d = d[['features']]
print len(d)

124011


In [17]:
d.dtypes

features    object
dtype: object

In [18]:
d.features.head()

id
0       [Elevator, Laundry in Building, Laundry in Uni...
1                   [Pre-War, Dogs Allowed, Cats Allowed]
10                                                     []
100                           [Doorman, Elevator, No Fee]
1000    [Roof Deck, Balcony, Elevator, Laundry in Buil...
Name: features, dtype: object

# Clean Features

In [19]:
d.features = d.features.apply(lambda fl: [f.lower().replace('-', '').replace(' ', '_') for f in fl])

# Number of Features

In [20]:
d['num_features'] = d.features.apply(lambda f: len(f)).astype(float64)

# Most Frequent Features

In [21]:
flat_features = [f for apt in d.features.tolist() for f in apt]
flat_feature_counts = Counter(flat_features)
flat_feature_counts.most_common(20)

[(u'elevator', 65833),
 (u'cats_allowed', 59194),
 (u'hardwood_floors', 59155),
 (u'dogs_allowed', 55207),
 (u'doorman', 52505),
 (u'dishwasher', 52036),
 (u'laundry_in_building', 47483),
 (u'no_fee', 45450),
 (u'fitness_center', 33420),
 (u'prewar', 26544),
 (u'laundry_in_unit', 23752),
 (u'roof_deck', 16466),
 (u'outdoor_space', 13414),
 (u'dining_room', 12847),
 (u'high_speed_internet', 10622),
 (u'balcony', 7793),
 (u'swimming_pool', 7153),
 (u'new_construction', 6457),
 (u'terrace', 5707),
 (u'exclusive', 5470)]

In [22]:
top_10_features = set([fc[0] for fc in flat_feature_counts.most_common(10)])
d['top_10_feature_coverage'] = d.features.apply(
    lambda fs: len(top_10_features.intersection(set(fs)))/10.0)
d['top_10_feature_ratio'] = d.features.apply(
    lambda fs: len(top_10_features.intersection(set(fs)))/(1.0 * len(fs)) if len(fs) > 0 else 0)

In [23]:
d.head(10)

,features,num_features,top_10_feature_coverage,top_10_feature_ratio
id,,,,
0,"[elevator, laundry_in_building, laundry_in_uni...",6.0,0.4,0.666667
1,"[prewar, dogs_allowed, cats_allowed]",3.0,0.3,1.000000
10,[],0.0,0.0,0.000000
100,"[doorman, elevator, no_fee]",3.0,0.3,1.000000
1000,"[roof_deck, balcony, elevator, laundry_in_buil...",10.0,0.4,0.400000
10000,"[doorman, elevator, fitness_center, cats_allow...",5.0,0.5,1.000000
100000,"[common_outdoor_space, cats_allowed, dogs_allo...",14.0,1.0,0.714286
100001,"[fireplace, dining_room, doorman, elevator, la...",11.0,0.8,0.727273
100002,"[hardwood_floors, new_construction]",2.0,0.1,0.500000


# Word Counts

In [24]:
d['features_s'] = d.features.apply(lambda fs: ' '.join(fs))

In [25]:
NUM_COUNT_FEATURES = 50
cv = CountVectorizer(stop_words='english', max_features=NUM_COUNT_FEATURES)
cv.fit(d.features_s)
voc = {v: k for k, v in cv.vocabulary_.iteritems()}
dc = pd.DataFrame(cv.transform(d.features_s).todense())
dc.columns = [('features_count_' + voc[i]) for i in range(NUM_COUNT_FEATURES)]
for c in dc.columns:
    d[c] = dc[c]

In [26]:
d.head(20).T

id,0,1,10,100,1000,10000,100000,100001,100002,100003,100004,100005,100006,100007,100008,100009,10001,100010,100011,100012
features,"[elevator, laundry_in_building, laundry_in_uni...","[prewar, dogs_allowed, cats_allowed]",[],"[doorman, elevator, no_fee]","[roof_deck, balcony, elevator, laundry_in_buil...","[doorman, elevator, fitness_center, cats_allow...","[common_outdoor_space, cats_allowed, dogs_allo...","[fireplace, dining_room, doorman, elevator, la...","[hardwood_floors, new_construction]","[fireplace, prewar, laundry_in_building, dishw...","[laundry_in_building, dishwasher, hardwood_flo...","[dogs_allowed, cats_allowed]","[loft, hardwood_floors]","[hardwood_floors, no_fee]","[cats_allowed, dogs_allowed]","[dining_room, doorman, elevator, garden/patio,...","[laundry_in_unit, private_outdoor_space, no_fee]","[doorman, prewar]","[prewar, dogs_allowed, cats_allowed]","[roof_deck, doorman, elevator, prewar, laundry..."
num_features,6,3,0,3,10,5,14,11,2,5,4,2,2,2,2,10,3,2,3,5
top_10_feature_coverage,0.4,0.3,0,0.3,0.4,0.5,1,0.8,0.1,0.4,0.3,0.2,0.1,0.2,0.2,0.8,0.1,0.2,0.3,0.4
top_10_feature_ratio,0.666667,1,0,1,0.4,1,0.714286,0.727273,0.5,0.8,0.75,1,0.5,1,1,0.8,0.333333,1,1,0.8
features_s,elevator laundry_in_building laundry_in_unit d...,prewar dogs_allowed cats_allowed,,doorman elevator no_fee,roof_deck balcony elevator laundry_in_building...,doorman elevator fitness_center cats_allowed d...,common_outdoor_space cats_allowed dogs_allowed...,fireplace dining_room doorman elevator laundry...,hardwood_floors new_construction,fireplace prewar laundry_in_building dishwashe...,laundry_in_building dishwasher hardwood_floors...,dogs_allowed cats_allowed,loft hardwood_floors,hardwood_floors no_fee,cats_allowed dogs_allowed,dining_room doorman elevator garden/patio prew...,laundry_in_unit private_outdoor_space no_fee,doorman prewar,prewar dogs_allowed cats_allowed,roof_deck doorman elevator prewar laundry_in_b...
features_count_balcony,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
features_count_cats_allowed,0,1,0,0,1,1,1,0,0,0,0,0,0,0,1,0,1,0,0,1
features_count_common_outdoor_space,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
features_count_dining_room,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
features_count_dishwasher,1,0,1,0,0,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0


# Save the Data

In [27]:
del d['features']
del d['features_s']

In [28]:
d.to_csv('tmp/features_features.csv')